In [1]:
#Attention mechanism
import torch
import torch.nn.functional as F


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X=torch.tensor([ [ [1.0,1.0,0.0,0.0,0 ],[-1.0,-1.0,0.0,0.0,0], [1.0,1.0,1.0,0.0,1] ],[[1.0,1.0,0.0,0.0,0 ],[-1.0,-1.0,0.0,0.0,0], [1.0,1.0,1.0,0.0,1] ]])

print(X.shape)
print(X)

B,N,D = X.shape
K=X
Q=X
V=X

energy = torch.einsum('bqd, bkd -> bqk', Q, K)

print(energy.shape)
print(energy)



torch.Size([2, 3, 5])
tensor([[[ 1.,  1.,  0.,  0.,  0.],
         [-1., -1.,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  0.,  0.],
         [-1., -1.,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  0.,  1.]]])
torch.Size([2, 3, 3])
tensor([[[ 2., -2.,  2.],
         [-2.,  2., -2.],
         [ 2., -2.,  4.]],

        [[ 2., -2.,  2.],
         [-2.,  2., -2.],
         [ 2., -2.,  4.]]])


In [3]:


att = F.softmax(energy, dim=-1)

print(att.shape)
print(att)


torch.Size([2, 3, 3])
tensor([[[0.4955, 0.0091, 0.4955],
         [0.0177, 0.9647, 0.0177],
         [0.1189, 0.0022, 0.8789]],

        [[0.4955, 0.0091, 0.4955],
         [0.0177, 0.9647, 0.0177],
         [0.1189, 0.0022, 0.8789]]])


In [4]:
out = torch.einsum('bal, blv -> bav ', att, V)

print(out.shape)
print(V)
print(out)


torch.Size([2, 3, 5])
tensor([[[ 1.,  1.,  0.,  0.,  0.],
         [-1., -1.,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  0.,  1.]],

        [[ 1.,  1.,  0.,  0.,  0.],
         [-1., -1.,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  0.,  1.]]])
tensor([[[ 0.9819,  0.9819,  0.4955,  0.0000,  0.4955],
         [-0.9293, -0.9293,  0.0177,  0.0000,  0.0177],
         [ 0.9956,  0.9956,  0.8789,  0.0000,  0.8789]],

        [[ 0.9819,  0.9819,  0.4955,  0.0000,  0.4955],
         [-0.9293, -0.9293,  0.0177,  0.0000,  0.0177],
         [ 0.9956,  0.9956,  0.8789,  0.0000,  0.8789]]])


In [6]:
## MultiHeadAttention

from einops import rearrange

x=torch.randn(32,64,512)
print(x.shape)
B,N,D=x.shape
H=8
x=torch.nn.Linear(D, D * 3)(x)

print(x.shape)


torch.Size([32, 64, 512])
torch.Size([32, 64, 1536])


In [7]:

#qkv = rearrange(self.qkv(x), "b n (h d qkv) -> (qkv) b h n d", h=self.num_heads, qkv=3)

print("x :")
x = rearrange(x, "b n (h d qkv) -> b n h d qkv", h=H, qkv=3)
print(x.shape)

print("qkv :")
qkv = rearrange(x, "b h n d qkv -> qkv b h n d")
print(qkv.shape)

print(qkv[0].shape)
print(qkv[1].shape)
print(qkv[2].shape)

print("Correlation, attention energy:")
energy = torch.einsum('bhqd, bhkd -> bhqk', qkv[0], qkv[1]) 
print(energy.shape)


x :
torch.Size([32, 64, 8, 64, 3])
qkv :
torch.Size([3, 32, 64, 8, 64])
torch.Size([32, 64, 8, 64])
torch.Size([32, 64, 8, 64])
torch.Size([32, 64, 8, 64])
Correlation, attention energy:
torch.Size([32, 64, 8, 8])
